In [1]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from time import time
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score



In [2]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [3]:
# PARTE 2 # 
######################
#Proponha um modelo para determinar se um anúncio receberá lead.
#Que outras informações, as quais você não teve acesso, poderiam ajudar a chegar a
#conclusões melhores?
########################

# Para o desenvolvimento da parte dois deste case, será utilizado um modelo de classificação
# para definir se o anúncio vai receber algum lead, mesmo que seja apenas 1.

###########

# Primeiro passo: Limpeza dos dados.

###########
df = pd.read_csv('dados.csv', sep=',')

print(df)

#  Removendo linhas com valores nulos
df = df.dropna()
# plotando boxplot para analisar visualmente alguns valores que fogem da distribuição normal dos dados para os dados discretos

'''



df.boxplot(column=['cod_anuncio', 'cod_cliente', 'cod_tipo_pessoa', 'prioridade', 'leads',
       'views', 'cliques_telefone*', 'cod_marca_veiculo', 'cod_modelo_veiculo',
       'cod_versao_veiculo', 'ano_modelo', 'cep_2dig',
       'qtd_fotos', 'km_veiculo', 'vlr_mercado'], rot=90)
'''

       cod_anuncio  cod_cliente  cod_tipo_pessoa  prioridade  leads  views  \
0                2         7941                2           3      1      0   
1                6         1770                1           3      1      0   
2                8          709                1           3      4      0   
3               13         9708                1           2     12      0   
4               15         1363                1           2      8      0   
...            ...          ...              ...         ...    ...    ...   
48660       195918         3065                2           3      1  34184   
48661       195924        12535                2           3      1  45048   
48662       195930         2284                2           3      1  71020   
48663       195934         2284                2           3      0  82940   
48664       195939         2284                2           3      0  93608   

       cliques_telefone*  cod_marca_veiculo  cod_modelo_veiculo

"\n\n\n\ndf.boxplot(column=['cod_anuncio', 'cod_cliente', 'cod_tipo_pessoa', 'prioridade', 'leads',\n       'views', 'cliques_telefone*', 'cod_marca_veiculo', 'cod_modelo_veiculo',\n       'cod_versao_veiculo', 'ano_modelo', 'cep_2dig',\n       'qtd_fotos', 'km_veiculo', 'vlr_mercado'], rot=90)\n"

In [4]:
# é visto por meio do boxplot que na colunakm_veiculo temos valores de quilometragem muito alto e que não faze sentido,
#esses valores serão removidos
mask = (df['km_veiculo']> 250000)
df= df.loc[~mask]
'''
df.boxplot(column=['cod_anuncio', 'cod_cliente', 'cod_tipo_pessoa', 'prioridade', 'leads',
       'views', 'cliques_telefone*', 'cod_marca_veiculo', 'cod_modelo_veiculo',
       'cod_versao_veiculo', 'ano_modelo', 'cep_2dig',
       'qtd_fotos', 'km_veiculo', 'vlr_mercado'], rot=90)
'''
# aqui os boxplot fazem mais sentido, indicando a distribuição dos dados
# apenas para analise vou plotar um boxplot a respeito do valor de mercado
#df.boxplot(column=['vlr_mercado'], rot=90), onde é possivel observar que temos um veiculo que passa de 1milhão de reais

# apenas para analise vou plotar um boxplot a respeito do número de leads
#df.boxplot(column=['leads'], rot=90)

# repplace valores iguais a -1 com 0
df = df.replace(-1, np.nan)
df = df.dropna()
df

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,arquente,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion
9,31,13771,1,2,16,0,8,2,1143,337123,...,N,N,N,N,N,S,S,S,N,N
16,55,15375,1,1,104,0,72,16,690,342914,...,S,S,S,S,S,S,S,S,N,N
25,95,16462,1,1,0,0,6,12,2722,342061,...,S,S,S,S,S,S,S,S,S,S
28,110,1987,1,1,0,0,2,2,1161,339241,...,S,N,S,S,S,S,S,N,N,N
30,117,18367,1,1,0,0,10,3,687,344077,...,S,N,S,N,S,S,S,S,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48602,37082,22723,1,2,52,52,35,2,670,337726,...,S,N,S,N,S,S,S,N,N,N
48609,39620,32376,1,1,56,56,53,17,2883,341540,...,S,S,S,S,S,S,S,S,S,S
48611,39761,14595,1,3,56,56,20,8,654,345546,...,S,S,S,S,S,S,S,S,S,S
48644,43573,24368,1,1,64,64,64,5,2966,343235,...,S,S,S,S,S,S,S,S,S,S


In [5]:
# criando um dataframe com os dados limpos

df_cleaned = df

# aqui vou criar dois dataframes, ou seja, dois datasets para criar o modelo.
# um dataset que carregara todos os atributos da base de dados (df_all), 
#e outro apenas com os atributos que foram selecionados para análise (df_sel)
df_all = df_cleaned
df_sel = df_cleaned[['leads', 'cod_marca_veiculo', 'cod_modelo_veiculo', 'ano_modelo', 'uf', 'cidade', 'km_veiculo']]


In [6]:
# Pré processamento dos dados
# Para podermos dizer se um anúncio receberá ou não leads, devemos considerar esse um problema de classificação binário
# , ouseja, um problema de sim ou não (1 e 0)

# Primeiro passo aqui é reformular a base de dados para o problema.
# Primeiramente vamos normalizar/transformar as variáveis categóricas em números discretos

df_all_cat = MultiColumnLabelEncoder(columns = df_all.columns).fit_transform(df_all)
df_sel_cat = MultiColumnLabelEncoder(columns = df_sel.columns).fit_transform(df_sel)
type(df_all_cat.values)

numpy.ndarray

In [7]:
df_all_cat

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,arquente,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion
9,0,9219,0,1,5,0,8,0,84,217,...,0,0,0,0,0,1,1,1,0,0
16,1,9545,0,0,27,0,72,9,10,1031,...,1,1,1,1,1,1,1,1,0,0
25,2,9789,0,0,0,0,6,7,191,832,...,1,1,1,1,1,1,1,1,1,1
28,3,937,0,0,0,0,2,0,90,456,...,1,0,1,1,1,1,1,0,0,0
30,4,10179,0,0,0,0,10,1,7,1391,...,1,0,1,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48602,6968,10988,0,1,14,14,35,0,5,270,...,1,0,1,0,1,1,1,0,0,0
48609,7512,12849,0,0,15,15,53,10,220,731,...,1,1,1,1,1,1,1,1,1,1
48611,7537,9390,0,2,15,15,20,4,2,1989,...,1,1,1,1,1,1,1,1,1,1
48644,8314,11307,0,0,17,17,64,3,237,1134,...,1,1,1,1,1,1,1,1,1,1


In [8]:
# agora criaremos os labels para os anuncios com leads, se leads for igual 0 o label é 0 se leads maior que 0 o label é 1
df_all_cat.loc[(df_all_cat.leads > 0), 'leads'] = 1
df_sel_cat.loc[(df_sel_cat.leads > 0), 'leads'] = 1

# Alógica aqui é: se o anuncio recebeu algum lead ele é 1 se não é igual a 0

In [9]:
#Criando dataframes para rodar os modelos
df_all_cat_target =df_all_cat[['leads']]

df_all_cat_input = df_all_cat[['cod_anuncio', 'cod_cliente', 'cod_tipo_pessoa', 'prioridade',
       'views', 'cliques_telefone*', 'cod_marca_veiculo', 'cod_modelo_veiculo',
       'cod_versao_veiculo', 'ano_modelo', 'cep_2dig', 'uf', 'cidade',
       'qtd_fotos', 'km_veiculo', 'vlr_mercado', 'flg_unico_dono',
       'flg_licenciado', 'flg_ipva_pago', 'flg_todas_revisoes_concessionaria',
       'flg_todas_revisoes_agenda_veiculo', 'flg_garantia_fabrica',
       'flg_blindado', 'flg_aceita_troca', 'flg_adaptado_pcd', 'combustivel',
       'cambio', 'portas', 'alarme', 'airbag', 'arquente', 'bancocouro',
       'arcondic', 'abs', 'desembtras', 'travaeletr', 'vidroseletr',
       'rodasliga', 'sensorchuva', 'sensorestacion']]




In [ ]:
# Agora com os dados limpos e pré processados é possível criar os modelos

y = df_all_cat_target
X = df_all_cat_input

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=66)

classifiers = {
    "Dummy"        : DummyClassifier(strategy='uniform'),
    "KNN(3)"       : KNeighborsClassifier(3), 
    "RBF SVM"      : SVC(gamma=2, C=1), 
    "Decision Tree": DecisionTreeClassifier(max_depth=7), 
    "Random Forest": RandomForestClassifier(max_depth=7, n_estimators=10, max_features=4), 
    "Neural Net"   : MLPClassifier(alpha=1), 
    "AdaBoost"     : AdaBoostClassifier(),
    "Naive Bayes"  : GaussianNB(), 
    "QDA"          : QuadraticDiscriminantAnalysis(),
    "Linear SVC"   : LinearSVC(),
    "Linear SVM"   : SVC(kernel="linear"), 
    "Gaussian Proc": GaussianProcessClassifier(1.0 * RBF(1.0)),
    "LogisticRegr" : LogisticRegression()
}

nfast = 14      # Don't run the very slow ones
head = list(classifiers.items())[:nfast]

for name, classifier in head:
    start = time()
    classifier.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    score = classifier.score(X_test, y_test)
    score_time = time()-start
    y_pred = classifier.predict(X_test)
    print(classification_report(y_test,y_pred))
    print("{:<15}| score = {:.3f} | time = {:,.3f}s/{:,.3f}s".format(name, score, train_time, score_time))

              precision    recall  f1-score   support

           0       0.22      0.50      0.31      2297
           1       0.78      0.51      0.62      8150

    accuracy                           0.51     10447
   macro avg       0.50      0.51      0.46     10447
weighted avg       0.66      0.51      0.55     10447

Dummy          | score = 0.493 | time = 0.000s/0.000s


<ipython-input-11-777b85c50193>:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  classifier.fit(X_train, y_train)


              precision    recall  f1-score   support

           0       0.24      0.13      0.17      2297
           1       0.78      0.88      0.83      8150

    accuracy                           0.71     10447
   macro avg       0.51      0.51      0.50     10447
weighted avg       0.66      0.71      0.68     10447

KNN(3)         | score = 0.714 | time = 0.371s/0.540s


C:\Users\abdal\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\abdal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2297
           1       0.78      1.00      0.88      8150

    accuracy                           0.78     10447
   macro avg       0.39      0.50      0.44     10447
weighted avg       0.61      0.78      0.68     10447

RBF SVM        | score = 0.780 | time = 365.151s/30.964s
              precision    recall  f1-score   support

           0       0.30      0.01      0.02      2297
           1       0.78      0.99      0.87      8150

    accuracy                           0.78     10447
   macro avg       0.54      0.50      0.45     10447
weighted avg       0.67      0.78      0.69     10447

Decision Tree  | score = 0.777 | time = 0.207s/0.006s


...

              precision    recall  f1-score   support

           0       0.25      0.00      0.00      2297
           1       0.78      1.00      0.88      8150

    accuracy                           0.78     10447
   macro avg       0.52      0.50      0.44     10447
weighted avg       0.66      0.78      0.68     10447

AdaBoost       | score = 0.780 | time = 1.991s/0.100s
              precision    recall  f1-score   support

           0       0.26      0.45      0.33      2297
           1       0.80      0.64      0.71      8150

    accuracy                           0.60     10447
   macro avg       0.53      0.54      0.52     10447
weighted avg       0.68      0.60      0.63     10447

Naive Bayes    | score = 0.597 | time = 0.025s/0.012s
              precision    recall  f1-score   support

           0       0.25      0.65      0.37      2297
           1       0.82      0.46      0.59      8150

    accuracy                           0.50     10447
   macro avg       0

C:\Users\abdal\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\abdal\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)



QDA            | score = 0.504 | time = 0.099s/0.014s


C:\Users\abdal\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\abdal\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\abdal\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


              precision    recall  f1-score   support

           0       0.23      0.00      0.00      2297
           1       0.78      1.00      0.88      8150

    accuracy                           0.78     10447
   macro avg       0.51      0.50      0.44     10447
weighted avg       0.66      0.78      0.68     10447

Linear SVC     | score = 0.779 | time = 5.590s/0.004s


In [ ]:


y = df_sel_cat[['leads']]
X = df_sel_cat[['cod_marca_veiculo', 'cod_modelo_veiculo', 'ano_modelo', 'uf', 'cidade', 'km_veiculo']]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=66)

classifiers = {
    "Dummy"        : DummyClassifier(strategy='uniform'),
    "KNN(3)"       : KNeighborsClassifier(3), 
    "RBF SVM"      : SVC(gamma=2, C=1), 
    "Decision Tree": DecisionTreeClassifier(max_depth=7), 
    "Random Forest": RandomForestClassifier(max_depth=7, n_estimators=10, max_features=4), 
    "Neural Net"   : MLPClassifier(alpha=1), 
    "AdaBoost"     : AdaBoostClassifier(),
    "Naive Bayes"  : GaussianNB(), 
    "QDA"          : QuadraticDiscriminantAnalysis(),
    "Linear SVC"   : LinearSVC(),
    "Linear SVM"   : SVC(kernel="linear"), 
    "Gaussian Proc": GaussianProcessClassifier(1.0 * RBF(1.0)),
    "LogisticRegr" : LogisticRegression()
}

nfast = 14      # Don't run the very slow ones
head = list(classifiers.items())[:nfast]

for name, classifier in head:
    start = time()
    classifier.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    score = classifier.score(X_test, y_test)
    score_time = time()-start
    y_pred = classifier.predict(X_test)
    print(classification_report(y_test,y_pred))
    print("{:<15}| score = {:.3f} | time = {:,.3f}s/{:,.3f}s".format(name, score, train_time, score_time))

In [ ]:
#Que outras informações, as quais você não teve acesso, poderiam ajudar a chegar aconclusões melhores?
# Talvez as datas de inclusão dos anúncios e também a cor dos veículos